In [1]:
!pip install beautifulsoup4==4.12.2
!pip install PyMuPDFb==1.24.10
!pip install tqdm

# Download 56-1 file

In [2]:
import requests
from bs4 import BeautifulSoup
import os
import zipfile
from tqdm import tqdm


ROOT = "downloaded-56-1"
SYMBOL = "KBANK"  # Change this symbol
FILE_PATH = f"{ROOT}/{SYMBOL}"
PATH_TO_TARGET = f"{FILE_PATH}/{SYMBOL}.zip"
session = requests.Session()



os.makedirs(FILE_PATH, exist_ok=True)
print("Searching the balance sheet file")

response = session.get(
    f"https://market.sec.or.th/public/idisc/th/Viewmore/fs-r561?searchSymbol={SYMBOL}",
)
soup = BeautifulSoup(response.text)
url = soup.find_all("tr")[1].find_all("td")[-1].find("a")["href"]

print(f"Found at {url}")
# Sizes in bytes.
total_size = int(response.headers.get("content-length", 0))
block_size = 1024
response = session.get(url, stream=True)

print("Downloading...")
with tqdm(total=total_size, unit="B", unit_scale=True) as progress_bar:
    with open(PATH_TO_TARGET, "wb") as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)

    
# with open(PATH_TO_TARGET, "wb") as f:
#     f.write(requests.get(url).content)

with zipfile.ZipFile(PATH_TO_TARGET, 'r') as zip_ref:
    zip_ref.extractall(FILE_PATH)
print("Done")

Searching the balance sheet file
Found at https://market.sec.or.th/public/idisc/Download?FILEID=dat/f56/0016ONE120320242034060731T.zip
Downloading...


11.4MB [00:01, 5.72MB/s]                                                          


Done


In [9]:
response = session.get(url, headers=headers, stream=True)

print("Downloading...")
with tqdm(total=total_size, unit="B", unit_scale=True) as progress_bar:
    with open(PATH_TO_TARGET, "wb") as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)

    
# with open(PATH_TO_TARGET, "wb") as f:
#     f.write(requests.get(url).content)

with zipfile.ZipFile(PATH_TO_TARGET, 'r') as zip_ref:
    zip_ref.extractall(FILE_PATH)
print("Done")

Downloading...


11.4MB [00:01, 10.2MB/s]                                                          


Done


# Extract table of contents

In [7]:
import fitz

def to_html(filepath: str):
    doc = fitz.open(filepath)
    for i, page in enumerate(doc):
        text = page.get_text("html")
        with open(f"pymupdf-page-{i}.html", "w") as fp:
            fp.write(text)
    doc.close()
    
doc = fitz.open(f'{FILE_PATH}/STRUCTURE{SYMBOL}T.PDF')
doc.get_toc()

[[1, 'Cover', 1],
 [1, 'TableContent', 2],
 [1, 'Bookmark', 3],
 [1, 'StructureAndOperation', 4],
 [1, 'Bookmark', 7],
 [1, 'RiskManagement', 8],
 [1, 'Bookmark', 11],
 [1, 'ESG', 12],
 [1, 'Bookmark', 15],
 [1, 'CorporateGovernancePolicy', 16],
 [1, 'Bookmark', 17],
 [1, 'CorporateGovernanceStructure', 18],
 [1, 'Bookmark', 30],
 [1, 'ResultOfCorporateGovernance', 31]]

# Extract tables

In [38]:
table = doc[5].find_tables()[1]
# print(doc.get_page_text(5))
table.to_pandas()

,รายการ (หน่วย : พันบาท),2564,2565,2566
0,ค่าใช้จ่ายการทำวิจัยและพัฒนา (R&D) ในระยะเวลา ...,N/A,N/A,N/A


In [39]:
table = doc[13].find_tables()[1]
# print(doc.get_page_text(5))
table.to_pandas()

,• การฝึกอบรมและพัฒนาพนักงาน,Col1,Col2,Col3
0,,2564,2565,2566
1,จำนวนชั่วโมงอบรมเฉลี่ยของพนักงาน (ชั่วโมง/คน/ปี),37.40,40.40,34.50


In [47]:
table = doc[17].find_tables()[1]
# print(doc.get_page_text(5))
table.to_pandas()

,รายชื่อกรรมการชุดปัจจุบัน,Col1,Col2,Col3
0,ข้อมูลทั่วไป,ตำแหน่งของกรรมการ,วันเริ่มดำรงตำแหน่ง,ประสบการณ์และความชำนาญ
1,1. นาง กอบกาญจน์ วัฒนวรางกูร\nเพศ: หญิง\nอายุ:...,"ประธานกรรมการ\n(กรรมการที่ไม่เป็นผู้บริหาร,\nก...",4 เม.ย. 2561,"ความยั่งยืน, การจัดการกลยุทธ์, การ\nจัดการความ..."
2,2. นางสาว สุจิตพรรณ ล่ำซำ\nเพศ: หญิง\nอายุ: 64...,รองประธานกรรมการ\n(กรรมการที่ไม่เป็นผู้บริหาร)...,30 เม.ย. 2541,"การเงิน, ความยั่งยืน, ธนาคาร, การ\nจัดการกลยุท..."
